# Machine Learning in Python

I document this file that used to educate myself some useful knowledge in Machine Learning.\~\
I code everything in Microsoft VS. To download Python packages, one should open the Windows prompt and type "py -m pip install PackageName"

"numpy" is the pacakage that used to create array and matrix.
"pandas" is commonly used to analyze the data.

## Linear regression

### Machine learning aspect

In [1]:
### import packages
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm


use 'read_csv' from 'pandas' to read csv data file.

In [2]:
df_origin = pd.read_csv("data/trade.csv")
df_origin

,iso3_o,iso3_d,contig,dist,comlang_off,tradeflow_comtrade_d
0,ABW,ABW,0.0,5.22500,0.0,NaN
1,ABW,AFG,0.0,13257.81400,0.0,NaN
2,ABW,AGO,0.0,9516.91310,0.0,NaN
3,ABW,AIA,0.0,983.26801,0.0,NaN
4,ABW,ALB,0.0,9091.74220,0.0,NaN
...,...,...,...,...,...,...
61499,ZWE,YMD,NaN,NaN,NaN,NaN
61500,ZWE,YUG,NaN,NaN,NaN,NaN
61501,ZWE,ZAF,1.0,2186.20610,1.0,170196.063
61502,ZWE,ZMB,1.0,396.80399,1.0,74451.109


In [3]:
df = df_origin.dropna(axis = 0, how = "any") # drop missing values 
df

,iso3_o,iso3_d,contig,dist,comlang_off,tradeflow_comtrade_d
7,ABW,ARE,0.0,12735.01200,0.0,538.973
8,ABW,ARG,0.0,5396.22020,1.0,0.052
15,ABW,AUT,0.0,8719.12990,0.0,109.376
18,ABW,BEL,0.0,7847.06980,1.0,29.035
23,ABW,BGR,0.0,9348.03910,0.0,1.957
...,...,...,...,...,...,...
61488,ZWE,UZB,0.0,7664.58110,0.0,2497.368
61490,ZWE,VCT,0.0,10699.65500,1.0,0.406
61494,ZWE,VNM,0.0,9231.17770,0.0,15669.277
61501,ZWE,ZAF,1.0,2186.20610,1.0,170196.063


In [4]:
# Drop only those rows where the specified column has a missing value
#df.dropna(subset=['varName'])

In [5]:
df["logtrade"] = np.log(df["tradeflow_comtrade_d"]);
df.describe()


C:\Users\niejun\AppData\Local\Temp\ipykernel_22316\989552665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["logtrade"] = np.log(df["tradeflow_comtrade_d"]);


,contig,dist,comlang_off,tradeflow_comtrade_d,logtrade
count,25309.000000,25309.000000,25309.000000,2.530900e+04,25309.000000
mean,0.019163,7585.013063,0.148050,6.706055e+05,7.306952
std,0.137101,4454.782636,0.355157,6.715881e+06,4.596065
min,0.000000,6.686000,0.000000,1.000000e-03,-6.907755
25%,0.000000,4021.170900,0.000000,5.459900e+01,4.000016
50%,0.000000,7191.661100,0.000000,2.056650e+03,7.628834
75%,0.000000,10582.334000,0.000000,4.877500e+04,10.794973
max,1.000000,19951.160000,1.000000,5.257563e+08,20.080348


In [6]:
df.dtypes   # check each variable's type

iso3_o                   object
iso3_d                   object
contig                  float64
dist                    float64
comlang_off             float64
tradeflow_comtrade_d    float64
logtrade                float64
dtype: object

In [7]:
df["logdist"] = np.log(df["dist"]);

C:\Users\niejun\AppData\Local\Temp\ipykernel_22316\2819130948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["logdist"] = np.log(df["dist"]);


Suppose I just want to run a simple regression that dependent variable is the log of trade, and indepedent variables are the log of distance, language and border

Extract the log of distane and assigned as independent variable.

In [8]:
x = df["logdist"];
print(x)

7        9.452110
8        8.593454
15       9.073275
18       8.967895
23       9.142922
           ...   
61488    8.944365
61490    9.277967
61494    9.130342
61501    7.689923
61502    5.983442
Name: logdist, Length: 25309, dtype: float64


Extract the log of trade and assigned as dependent variable

In [9]:
y = df["logtrade"];
print(y)

7         6.289665
8        -2.956512
15        4.694791
18        3.368502
23        0.671413
           ...    
61488     7.822993
61490    -0.901402
61494     9.659457
61501    12.044706
61502    11.217898
Name: logtrade, Length: 25309, dtype: float64


In following code, I run this simple linear regression by using "OLS" feature from the package "statsmodels". The nice aspect of this package is it returns the table of results. However, I need to us "sm.add_constant" to add the constant $\beta_{0}$ in the regression model.

In [20]:
x = sm.add_constant(x);
model = sm.OLS(y,x);
results = model.fit();

results.summary()   # this return the estimated results

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                logdist   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.058e+33
Date:                Fri, 06 Sep 2024   Prob (F-statistic):               0.00
Time:                        16:54:34   Log-Likelihood:             8.2299e+05
No. Observations:               25309   AIC:                        -1.646e+06
Df Residuals:                   25307   BIC:                        -1.646e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.892e-15   1.23e-16     39.871      0.000    4.65e-15    5.13e-15
logdist        1.0000   1.41e-17   7.11e+16      0.000       1.000       1.000
==============================================================================
Omnibus:                    24068.644   Durbin-Watson:                   0.020
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           734894.613
Skew:                          -4.870   Prob(JB):                         0.00
Kurtosis:                      27.536   Cond. No.                         94.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
x.shape
y.shape

(25309,)

In [12]:
x = df.iloc[:, df.columns.get_loc("logdist") ].values
y = df.iloc[:, df.columns.get_loc("logtrade") ].values
x= x.reshape(-1, 1)

In [13]:
reg = LinearRegression().fit(x, y)

In [22]:
reg.coef_

array([-1.46101617])

In [21]:
reg.n_features_in_

1